In [1]:
import tensorflow as tf
import pandas as pd
import datetime
import numpy as np
import os
from tensorflow.python import pywrap_tensorflow

## Preprocessing

In [2]:
'''
PARAMETERS TO CHANGE
'''
stock_symbol = 'c'
train_model = True
test_model = True
''''''

training_data_dir = './training_data/'
y_columns = ['ppo', 'rsi']
#assert(train_model ^ test_model)
model_dir = "./regularizer_models/" + stock_symbol + '/'

if train_model:
    training_data = pd.read_csv(training_data_dir + stock_symbol + '_training_data.csv')
    unneeded_columns = ['window']

    x_columns = list(training_data.columns)
    for to_remove in (y_columns + unneeded_columns):
        x_columns.remove(to_remove)

    train_x = training_data[x_columns].values
    train_y = training_data[y_columns].values

    model_dir = "./regularizer_models/" + stock_symbol + '/'

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
if test_model:
    test_data = pd.read_csv(training_data_dir + stock_symbol + '_test_data.csv')
    
    unneeded_columns = ['Date_to_use_on']
    x_columns = list(test_data.columns)

    for to_remove in (unneeded_columns):
        x_columns.remove(to_remove)

    
    test_x = test_data[x_columns].values
    
    
    

## ANN Model

In [3]:
# Parameters
learning_rate = 0.1
num_epochs = 50000
batch_size = 128
display_epoch = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = len(x_columns) # Number of features
num_output = len(y_columns) # PPO, RSI parameters


def ANN(X):
    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_output]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_output]))
    }

    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)    
    
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    
    # Output fully connected layer with a neuron for each output
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out = tf.square(out_layer)
    
    return out

X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_output])

output = ANN(X)

# Loss & Optimizer
loss_op = tf.losses.mean_squared_error(Y, output, weights=1)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)
# Accuracy
accuracy =  tf.abs(tf.reduce_mean(tf.round(output) - Y))

In [4]:
if train_model:

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        # Run initializer
        sess.run(init)
        for epoch in range(num_epochs):
            sess.run(train_op, feed_dict={X: train_x, Y: train_y})

            if epoch % display_epoch == 0 or epoch == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_x, Y: train_y})

                print("Epoch " + str(epoch) + ", Loss= " + \
                      "{:.4f}".format(loss) + ", Training Error= " + \
                      "{:.3f}".format(acc))


        print(sess.run(output, feed_dict={X: train_x}))
        saver.save(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        
        
        print('Model Saved')

      

Epoch 0, Loss= 527105523712.0000, Training Error= 569576.438
Epoch 1, Loss= 823674816.0000, Training Error= 26432.018
Epoch 100, Loss= 26882.8711, Training Error= 83.490
Epoch 200, Loss= 14317.2344, Training Error= 54.336
Epoch 300, Loss= 8758.9229, Training Error= 42.128
Epoch 400, Loss= 6961.5430, Training Error= 37.131
Epoch 500, Loss= 5600.5503, Training Error= 33.043
Epoch 600, Loss= 4643.3652, Training Error= 29.864
Epoch 700, Loss= 3925.2563, Training Error= 27.202
Epoch 800, Loss= 3364.1450, Training Error= 24.912
Epoch 900, Loss= 2917.4702, Training Error= 22.930
Epoch 1000, Loss= 2564.4294, Training Error= 21.261
Epoch 1100, Loss= 2233.5203, Training Error= 19.770
Epoch 1200, Loss= 1952.1266, Training Error= 18.416
Epoch 1300, Loss= 1746.6244, Training Error= 17.327
Epoch 1400, Loss= 1579.1211, Training Error= 16.420
Epoch 1500, Loss= 1435.3488, Training Error= 15.634
Epoch 1600, Loss= 1317.8368, Training Error= 14.956
Epoch 1700, Loss= 1221.1279, Training Error= 14.386
Epoch

Epoch 16200, Loss= 22.6070, Training Error= 0.490
Epoch 16300, Loss= 22.3487, Training Error= 0.498
Epoch 16400, Loss= 22.0901, Training Error= 0.525
Epoch 16500, Loss= 21.8228, Training Error= 0.533
Epoch 16600, Loss= 21.6012, Training Error= 0.544
Epoch 16700, Loss= 21.3643, Training Error= 0.564
Epoch 16800, Loss= 21.1500, Training Error= 0.577
Epoch 16900, Loss= 20.9501, Training Error= 0.588
Epoch 17000, Loss= 20.7527, Training Error= 0.596
Epoch 17100, Loss= 20.5618, Training Error= 0.612
Epoch 17200, Loss= 20.3622, Training Error= 0.619
Epoch 17300, Loss= 20.1698, Training Error= 0.629
Epoch 17400, Loss= 19.9907, Training Error= 0.644
Epoch 17500, Loss= 19.8135, Training Error= 0.651
Epoch 17600, Loss= 19.6390, Training Error= 0.656
Epoch 17700, Loss= 19.4619, Training Error= 0.661
Epoch 17800, Loss= 19.2901, Training Error= 0.666
Epoch 17900, Loss= 19.1046, Training Error= 0.677
Epoch 18000, Loss= 18.9254, Training Error= 0.686
Epoch 18100, Loss= 18.7472, Training Error= 0.688


Epoch 32600, Loss= 19.5493, Training Error= 0.559
Epoch 32700, Loss= 19.4256, Training Error= 0.546
Epoch 32800, Loss= 19.3064, Training Error= 0.530
Epoch 32900, Loss= 19.1907, Training Error= 0.515
Epoch 33000, Loss= 19.0782, Training Error= 0.502
Epoch 33100, Loss= 18.9708, Training Error= 0.489
Epoch 33200, Loss= 18.8597, Training Error= 0.483
Epoch 33300, Loss= 18.7431, Training Error= 0.480
Epoch 33400, Loss= 18.6364, Training Error= 0.470
Epoch 33500, Loss= 18.5346, Training Error= 0.462
Epoch 33600, Loss= 18.4373, Training Error= 0.452
Epoch 33700, Loss= 18.3237, Training Error= 0.439
Epoch 33800, Loss= 18.2273, Training Error= 0.426
Epoch 33900, Loss= 18.1316, Training Error= 0.419
Epoch 34000, Loss= 18.0405, Training Error= 0.414
Epoch 34100, Loss= 17.9506, Training Error= 0.410
Epoch 34200, Loss= 17.8623, Training Error= 0.403
Epoch 34300, Loss= 17.7736, Training Error= 0.394
Epoch 34400, Loss= 17.6856, Training Error= 0.387
Epoch 34500, Loss= 17.5967, Training Error= 0.377


Epoch 49000, Loss= 18.4851, Training Error= 0.166
Epoch 49100, Loss= 18.4586, Training Error= 0.058
Epoch 49200, Loss= 18.4580, Training Error= 0.054
Epoch 49300, Loss= 18.4649, Training Error= 0.023
Epoch 49400, Loss= 18.4834, Training Error= 0.157
Epoch 49500, Loss= 18.4550, Training Error= 0.073
Epoch 49600, Loss= 18.4541, Training Error= 0.057
Epoch 49700, Loss= 18.4891, Training Error= 0.032
Epoch 49800, Loss= 18.4526, Training Error= 0.042
Epoch 49900, Loss= 18.4533, Training Error= 0.093
[[1.6417689 3.5444336]
 [1.8874831 3.0291078]
 [1.5248743 3.818388 ]
 ...
 [1.677586  3.4644413]
 [1.8773675 3.0488617]
 [1.5222211 3.824846 ]]
Model Saved


In [5]:
if test_model:
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        '''var_name_list = [v.name for v in tf.trainable_variables()]
        print(var_name_list)
        
        reader = pywrap_tensorflow.NewCheckpointReader(model_dir + stock_symbol + '_regularizer.ckpt')
        var_to_shape_map = reader.get_variable_to_shape_map()'''
        
        saver.restore(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        print("Model restored.")
        predicted = tf.round(sess.run(output, feed_dict={X : test_x})).eval()
        predicted = np.array(predicted)
        
    result = pd.DataFrame(predicted, columns = y_columns)
    result['Date_to_use_on'] = test_data['Date_to_use_on']
    result = result[['Date_to_use_on'] + y_columns]
    result.to_csv(training_data_dir+'c_test_parameters.csv', index=False)
    

INFO:tensorflow:Restoring parameters from ./regularizer_models/c/c_regularizer.ckpt
Model restored.


In [6]:
result

,Date_to_use_on,ppo,rsi
0,2009-11-20,2.0,3.0
1,2009-11-23,2.0,3.0
2,2009-11-24,2.0,3.0
3,2009-11-25,2.0,3.0
4,2009-11-27,2.0,3.0
5,2009-11-30,2.0,3.0
6,2009-12-01,2.0,3.0
7,2009-12-02,1.0,4.0
8,2009-12-03,1.0,4.0
9,2009-12-04,1.0,4.0
